Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
import numpy as np

In [ ]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [ ]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size= (NUM_KNAPSACKS, NUM_DIMENSIONS))

In [ ]:
# A random solution
solution = np.array(
    [np.random.random(NUM_ITEMS) < 0.5 for _ in range(NUM_KNAPSACKS)], dtype=np.bool
)

In [ ]:
solution

In [ ]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

In [ ]:
# Check if the solution is valid
all_knapsacks = np.any(solution, axis=0)
np.all(WEIGHTS[all_knapsacks].sum(axis=0) < CONSTRAINTS)

## TEST PROBLEMS

In [ ]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [ ]:
def evaluate(solution, values, weights, constraints):
    """
    Calculate the fitness of a solution
    fitness = total_value
    """

    total_value = 0

    for k in range(constraints.shape[0]): # for each knapsack
        items_in_k = np.where(solution== k)[0]
        if len(items_in_k) ==0:
            continue
    
        used = np.sum(weights[items_in_k], axis =0)
        if np.any(used> constraints[k]):
            return -np.inf 

        total_value += np.sum(values[items_in_k]) 
    
    return total_value

def random_solution(num_items, num_knapsacks):
    """Create a random solution, each item can be assigned to a knapsack (0 ..K-1) or not taken (-1)
    """
    return rng.integers(-1, num_knapsacks, size = num_items)

def tweak(solution, num_knapsacks):
    """
    Create a neighbour solution by changing a single item
    """
    new_sol= solution.copy()
    idx = rng.integers(0, len(solution)) # choose a random item
    # assign to a new knapsack (or remove it)
    new_sol[idx] = rng.integers(-1, num_knapsacks)
    return new_sol

In [ ]:
def hill_climbing(values, weights, constraints,num_knapsacks, interations = 5000, patience = 500):
    """ 
    Implementation of Hill Climbing (Random-Mutation Hill Climber).
    """
    current = random_solution(len(values), num_knapsacks)
    current_value = evaluate(current, values, weights, constraints)
    best = current.copy()
    best_value = current_value
    no_improve =0

    for step in range(interations):
        candidate = tweak(current, num_knapsacks)
        candidate_value = evaluate(candidate, values, weights, constraints)
        
        if candidate_value == -np.inf:
            continue

        #if it improves -> accept
        if candidate_value > current_value:
            current, current_value = candidate, candidate_value
            no_improve = 0

            # if its the best so far -> update best
            if current_value>best_value:
                best, best_value = current.copy(), current_value
        else:
            no_improve +=1
        
        #Stopping condition : no more improve
        if no_improve> patience:
            break
    
    return best, best_value


In [ ]:
""" Execution on problem 1"""
best_solution, best_fitness = hill_climbing(VALUES, WEIGHTS, CONSTRAINTS,
                                            NUM_KNAPSACKS,
                                            interations=5000, 
                                            patience= 500)

print("\n RESULT HILL CLIMBING")
print("Best value found:", best_fitness)
print("Best solution:", best_solution)

valid_constraints = True
for k in range(NUM_KNAPSACKS):
    items_in_k = np.where(best_solution == k)[0]
    if len(items_in_k) > 0:
        load_k = np.sum(WEIGHTS[items_in_k], axis=0)
        if np.any(load_k > CONSTRAINTS[k]):
            valid_constraints = False
            print(f"ERROR: Knapsack {k} is overloaded")
            break

print("\nThe solution found is valid:", valid_constraints)


In [ ]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [ ]:
def evaluate(solution, values, weights, constraints, num_knapsacks):
     """
    Calculate the fitness of a solution
    fitness = total_value - penalty_factor * penalty
    """ 
     total_value=0
     for k in range(num_knapsacks): #for each knapsack
          items_in_k = np.where(solution== k)[0]
          if len(items_in_k) ==0:
            continue
    
          used = np.sum(weights[items_in_k], axis =0)
          if np.any(used> constraints[k]):
            return -np.inf 

          total_value += np.sum(values[items_in_k]) 
    
     return total_value

def tweak(solution, tabu_list, num_knapsacks, rng):
     """ 
     Create a neighbor by modifying a no-tabu item
     """
     new_solution = solution.copy()
     
     item_to_move = rng.integers(0, len(solution))
     while item_to_move in tabu_list:
         item_to_move = rng.integers(0, len(solution))

     current_assignment = new_solution[item_to_move]
     new_assignment = current_assignment
     while new_assignment == current_assignment:
         new_assignment= rng.integers(-1, num_knapsacks)

     new_solution[item_to_move]= new_assignment
     return new_solution, item_to_move

def generate_feasible_solution(values, weights, constraints, num_knapsacks, rng):
    num_items = len(values)
    solution = np.full(num_items, -1)
    
    for i in rng.permutation(num_items):
        k = rng.integers(0, num_knapsacks)
        # prova a metterlo in uno zaino se non sfora
        used = np.sum(weights[solution == k], axis=0) if np.any(solution == k) else np.zeros(weights.shape[1])
        if np.all(used + weights[i] <= constraints[k]):
            solution[i] = k
    return solution

In [ ]:
def tabu_search(values, weights, constraints, num_knapsacks, num_items, iterations, neighborhood_size, tabu_tenure):
    """Implementation of Tabu Search's algorithm"""
    # 1. initialization
    current_solution = generate_feasible_solution(values, weights, constraints, num_knapsacks, rng)
    best_solution = current_solution
    best_fitness = evaluate(best_solution, values, weights, constraints, num_knapsacks)
    tabu_list=[]

    # 2. Main cycle
    print(f"Tabu Search execution for {iterations} iterations..")
    for _ in range(iterations):
        neighborhood_solutions = []
        neighborhood_moves= []
        for _ in range(neighborhood_size):
            neighbor_sol, neighbor_move = tweak(current_solution, tabu_list, num_knapsacks, rng)
            neighborhood_solutions.append(neighbor_sol)
            neighborhood_moves.append(neighbor_move)

        neighborhood_fitness = [evaluate(s, values, weights, constraints, num_knapsacks ) for s in neighborhood_solutions]
    
        # 3. Best neighbot selection
        best_neighbor_indx = np.argmax(neighborhood_fitness)
        # 4. update
        current_solution = neighborhood_solutions[best_neighbor_indx]
        current_fitness = neighborhood_fitness[best_neighbor_indx]
        move_made = neighborhood_moves[best_neighbor_indx]

        # 5. update tabu list
        tabu_list.append(move_made)
        if len(tabu_list)> tabu_tenure:
            tabu_list.pop(0)
        
        # 6. update global best
        if current_fitness> best_fitness:
            best_solution = current_solution
            best_fitness = current_fitness
    
    print("Search complete")
    return best_solution, best_fitness


def check_validity(solution, weights, constraints, num_knapstack ):
    for k in range(num_knapstack):
        items_in_k = np.where(solution == k)[0]
        if items_in_k.size > 0:
            load_k = np.sum(weights[items_in_k], axis=0)
            if np.any(load_k > constraints[k]):
                return False
    return True
    

In [ ]:
""" Execution on problem 2"""
best_solution, best_fitness = tabu_search(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, NUM_ITEMS,
                                          iterations= 2000, neighborhood_size = 100, tabu_tenure=15)
print("\n RESULT TABU SEARCH")
is_valid = check_validity(best_solution, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)
print("Best value found:", best_fitness)
print("Best solution:", best_solution)
print("\nThe solution found is valid:", is_valid)


In [ ]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [ ]:
def evaluate(solution, values, weights, constraints, num_knapsacks):
    penalty_factor = np.max(values) + 1
    total_value = np.sum(values[solution >= 0])
    penalty = 0
    for k in range(num_knapsacks):
        items_in_k = np.where(solution == k)[0]
        if items_in_k.size > 0:
            load_k = np.sum(weights[items_in_k], axis=0)
            violation = np.maximum(load_k - constraints[k], 0)
            penalty += np.sum(violation)
    return total_value - penalty_factor * penalty


def evaluate_move(current_fitness, solution, move, values, weights, constraints, num_knapsacks):
    """ Calculate new fitness differentially (incrementally), analyze only the impact of a single move
        The goal is to update the fitness without recalculating evrything from scratch
        new_fitness = current_fitness + delta
        delta = delta_value + delta_penalty
        delta_value = difference of value
        delta_penalty = difference of penalty
    """
    item_idx, old_k, new_k = move
    penalty_factor = np.max(values) + 1

    # 1. Calculating the delta value, the total value only changes if an item is added or removed.
    # If the item moves between two backpascks (k1 -> k2), the total value does not change
    delta_value = 0
    if old_k == -1 and new_k != -1:
        delta_value = values[item_idx]
    elif old_k != -1 and new_k == -1:
        delta_value = -values[item_idx]

    # 2. Calculating the delta penalty
    old_penalty = 0
    new_penalty = 0
    
    if old_k != -1:
        items_in_old_k = np.where(solution == old_k)[0]
        load_before = np.sum(weights[items_in_old_k], axis=0)
        load_after = load_before - weights[item_idx]
        old_penalty += np.sum(np.maximum(load_before - constraints[old_k], 0))
        new_penalty += np.sum(np.maximum(load_after - constraints[old_k], 0))

    if new_k != -1:
        items_in_new_k = np.where(solution == new_k)[0]
        load_before = np.sum(weights[items_in_new_k], axis=0)
        load_after = load_before + weights[item_idx]
        old_penalty += np.sum(np.maximum(load_before - constraints[new_k], 0))
        new_penalty += np.sum(np.maximum(load_after - constraints[new_k], 0))
        
    delta_penalty = new_penalty - old_penalty
    
    # 3. Calculating new fitness
    new_fitness = current_fitness + delta_value - (penalty_factor * delta_penalty)
    return new_fitness

def tabu_search_optimized(values, weights, constraints, num_knapsacks, num_items, iterations, neighborhood_size, tabu_tenure):
    
    rng = np.random.default_rng(seed=42)
    
    current_solution = rng.integers(-1, num_knapsacks, size=num_items)
    current_fitness = evaluate(current_solution, values, weights, constraints, num_knapsacks)
    
    best_solution_so_far = current_solution.copy()
    best_fitness_so_far = current_fitness
    
    tabu_queue = deque(maxlen=tabu_tenure)
    tabu_set = set()

    print(f"Tabu Search execution OPTIMIZED for {iterations} iterations...")
    for i in range(iterations):
        best_neighbor_fitness = -np.inf
        best_neighbor_move = None
        
        for _ in range(neighborhood_size):
            item_to_move = rng.integers(0, num_items)
            while item_to_move in tabu_set:
                item_to_move = rng.integers(0, num_items)
            
            old_assignment = current_solution[item_to_move]
            new_assignment = old_assignment
            while new_assignment == old_assignment:
                new_assignment = rng.integers(-1, num_knapsacks)
            
            move = (item_to_move, old_assignment, new_assignment)
        
            neighbor_fitness = evaluate_move(current_fitness, current_solution, move, values, weights, constraints, num_knapsacks)
            
            if neighbor_fitness > best_neighbor_fitness:
                best_neighbor_fitness = neighbor_fitness
                best_neighbor_move = move

        if best_neighbor_move is not None:
            item, old_k, new_k = best_neighbor_move
            current_solution[item] = new_k
            current_fitness = best_neighbor_fitness

            if len(tabu_queue) == tabu_tenure:
                item_to_remove = tabu_queue[0]
                tabu_set.remove(item_to_remove)
            tabu_queue.append(item)
            tabu_set.add(item)

            if current_fitness > best_fitness_so_far:
                best_fitness_so_far = current_fitness
                best_solution_so_far = current_solution.copy()

    return best_solution_so_far, best_fitness_so_far

def check_validity(solution, weights, constraints, num_knapstack ):
    for k in range(num_knapstack):
        items_in_k = np.where(solution == k)[0]
        if items_in_k.size > 0:
            load_k = np.sum(weights[items_in_k], axis=0)
            if np.any(load_k > constraints[k]):
                return False
    return True
    

In [ ]:
""" Execution on problem 3"""
from collections import deque
best_solution, best_fitness = tabu_search_optimized(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, NUM_ITEMS,
                                          iterations= 2000, neighborhood_size = 100, tabu_tenure=70)
print("\n RESULT TABU SEARCH")
is_valid = check_validity(best_solution, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)
print("Best value found:", best_fitness)
print("Best solution:", best_solution)
print("\snThe solution found is valid:", is_valid)
